In [19]:
import keras
import numpy as np
import os
import random
import keras 
from keras.layers import Input, Conv2D,GlobalAveragePooling2D, Dense, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model
from keras.layers import DepthwiseConv2D

from tensorflow_model_optimization.quantization.keras import vitis_quantize




shape_range =range(1024,2048,168)
for shape_x in shape_range:
    shape_y = shape_x
    project_name = "fcn_segment"

    channels = 1
    std_shape = (shape_x, shape_y, channels) # 输入尺寸, std_shape[0]: img_rows, std_shape[1]: img_cols
                                     # 这个尺寸按你的图像来, 如果你的图大小不一, 那 img_rows 和 image_cols
                                     # 都要设置成 None, 如果你在用 Generator 加载数据时有扩展边缘, 那 std_shape
                                     # 就是扩展后的尺寸

    img_input = keras.layers.Input(shape = std_shape, name = "input")
    
    filter1_num = random.randrange(16,32,2)
    conv_1 = keras.layers.Conv2D(filter1_num, kernel_size = (3, 3), activation = "relu",
                                 padding = "same", name = "conv_1")(img_input)
    max_pool_1 = keras.layers.MaxPool2D(pool_size = (2, 2), strides = (2, 2),
                                        name = "max_pool_1")(conv_1)

    filter2_num = random.randrange(filter1_num,64,2)
    conv_2 = keras.layers.Conv2D(filter2_num, kernel_size = (3, 3), activation = "relu",
                                 padding = "same", name = "conv_2")(max_pool_1)
    max_pool_2 = keras.layers.MaxPool2D(pool_size = (2, 2), strides = (2, 2),
                                        name = "max_pool_2")(conv_2)

    filter3_num = random.randrange(filter2_num,128,2)
    conv_3 = keras.layers.Conv2D(filter3_num, kernel_size = (3, 3), activation = "relu",
                                 padding = "same", name = "conv_3")(max_pool_2)
    max_pool_3 = keras.layers.MaxPool2D(pool_size = (2, 2), strides = (2, 2),
                                        name = "max_pool_3")(conv_3)

    filter4_num = random.randrange(filter3_num,256,2)
    conv_4 = keras.layers.Conv2D(filter4_num , kernel_size = (3, 3), activation = "relu",
                                 padding = "same", name = "conv_4")(max_pool_3)
    max_pool_4 = keras.layers.MaxPool2D(pool_size = (2, 2), strides = (2, 2),
                                        name = "max_pool_4")(conv_4)

    filter5_num = random.randrange(filter4_num,512,2)
    conv_5 = keras.layers.Conv2D(filter5_num, kernel_size = (3, 3), activation = "relu",
                                 padding = "same", name = "conv_5")(max_pool_4)
    max_pool_5 = keras.layers.MaxPool2D(pool_size = (2, 2), strides = (2, 2),
                                        name = "max_pool_5")(conv_5)
    # max_pool_5 转置卷积上采样 2 倍至 max_pool_4 一样大
    up6 = keras.layers.Conv2DTranspose(filter4_num , kernel_size = (3, 3),
                                       strides = (2, 2),
                                       padding = "same",
                                       kernel_initializer = "he_normal",
                                       name = "upsamping_6")(max_pool_5)

    _16s = keras.layers.add([max_pool_4, up6])

    # _16s 上采样 16 倍后与输入尺寸相同
    up7 = keras.layers.UpSampling2D(size = (16, 16), interpolation = "nearest",
                                    name = "upsamping_7")(_16s)

    # 这里 kernel 也是 3 * 3, 也可以同 FCN-32s 那样修改的
    conv_7 = keras.layers.Conv2D(1, kernel_size = (3, 3), activation = "sigmoid",
                                 padding = "same", name = "conv_7")(up7)
    X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_test=np.random.randint(0,10,size=[60,])
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0
    for step in range(1,6):
        model = keras.Model(img_input, conv_7, name = project_name)
        model.summary()
        model.save('./FCN_h5/FCN_{input_x}_{input_y}_{step}.h5'.format(input_x=shape_x,input_y=shape_y,step=step))
        quantizer = vitis_quantize.VitisQuantizer(model)
        quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:8], weight_bit=8, activation_bit=8)
        quantized_model.save('quantized.h5')
        !vai_c_tensorflow2 \
                        --model ./quantized.h5 \
                        --arch ./arch.json \
                        --output_dir ./xmodel_FCN \
                        --net_name deploy
        os.rename("./xmodel_FCN/deploy.xmodel","./xmodel_FCN/FCN_{input_x}_{input_y}_{step}.xmodel".format(input_x=shape_x, input_y=shape_y, step=step))
 



Model: "fcn_segment"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 1024, 1024,  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 1024, 1024, 2 260         input[0][0]                      
__________________________________________________________________________________________________
max_pool_1 (MaxPooling2D)       (None, 512, 512, 26) 0           conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 512, 512, 42) 9870        max_pool_1[0][0]                 
________________________________________________________________________________________

2022-04-30 08:02:12.779428: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 4932501504 exceeds 10% of free system memory.
2022-04-30 08:02:13.471125: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 4932501504 exceeds 10% of free system memory.
2022-04-30 08:02:13.471174: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 4932501504 exceeds 10% of free system memory.
2022-04-30 08:02:13.471195: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 4932501504 exceeds 10% of free system memory.
2022-04-30 08:02:13.471231: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 4932501504 exceeds 10% of free system memory.


1/1 [==============================] - 33s 33s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :100%|█| 39/39 [00:00<00:00, 3311.69it/s]            
[INFO] dump xmodel: /tmp/deploy_org.xmodel
[UNILOG][INFO] Compile mode: dpu
[UNILOG][INFO] Debug mode: function
[UNILOG][INFO] Target architecture: DPUCZDX8G_ISA0_B2304_MAX_BG2
[UNILOG][INFO] Graph name: fcn_segment, with op num: 67

FileNotFoundError: [Errno 2] No such file or directory: './xmodel_FCN/deploy.xmodel' -> './xmodel_FCN/FCN_1024_1024_1.xmodel'